# Input-output properties cells

In [1]:
import h5py
import json
import numpy as np
import os
import sys
from matplotlib import pyplot as plt

rootFolder = '/home/fernando/S1_mouse'
#------------------------------------------------------------------------------  

In [2]:
#------------------------------------------------------------------------------
# Cells
#------------------------------------------------------------------------------
# Load 55 Morphological Names-> L1:6 L23:10 L4:12 L5:13 L6:14
# Load 207 Morpho-electrical Names-> L1:14 L23:43 L4:46 L5:52 L6:52

with open(rootFolder + '/info/anatomy/S1-cells-distributions-Rat.txt') as metype_file:
    metype_content = metype_file.read()       

MtypeNumber = {}
MEtypeNumber = {}
MtypePop = []
MEtypePop = []
popLabel = {}
N = 0
for line in metype_content.split('\n')[:-1]:
    metype, mtype, etype, n, m = line.split()
    MEtypeNumber[metype] = int(n)
    popLabel[metype] = mtype
    MtypeNumber[mtype] = int(m)

    if mtype not in MtypePop:
        MtypePop.append(mtype)
    MEtypePop.append(metype)
    
    N = N + int(n)
    
print ('Number of cells = %d' % N)

Number of cells = 31346


In [3]:
Epops = ['L23_PC', 'L4_PC', 'L4_SS', 'L4_SP', 
             'L5_TTPC1', 'L5_TTPC2', 'L5_STPC', 'L5_UTPC',
             'L6_TPC_L1', 'L6_TPC_L4', 'L6_BPC', 'L6_IPC', 'L6_UTPC']
Ipops = []
for popName in MtypePop:
    if popName not in Epops:
        Ipops.append(popName)

In [4]:
# for cellName in MEtypePop:
#     for number in range[1,6]:
#         os.chdir('/home/fernando/Downloads/hoc_combos_syn.1_0_10.allzips/'+cellName+'_'+str(number)+'/')
#         !nrnivmodl mechanisms

In [5]:
def loadTemplateName(cellName,number): 
    f = open('/home/fernando/S1_BBP/cell_data/'+cellName+'_'+str(number)+'/template.hoc', 'r')
    for line in f.readlines():
        if 'begintemplate' in line:
            templatename = str(line)     
    templatename=templatename[:-1]        
    templatename=templatename[14:]
    return templatename

In [6]:
def loadCell(cellName,cellTemplateName,number):
    
    from neuron import h
    
    os.chdir('/home/fernando/S1_BBP/cell_data/'+cellName+'_'+str(number)+'/')
    h.load_file("stdrun.hoc")
    h.load_file('import3d.hoc')
    try:
        h.xopen("morphology.hoc")
    except:
        pass
    try:
        h.xopen("biophysics.hoc")
    except:
        pass
    try:
        h.xopen("synapses/synapses.hoc")
    except:
        pass
    try:
        h.xopen("template.hoc")
    except:
        pass
    
    cell = getattr(h, cellTemplateName)(0)
    
    i=0
    for secs in cell.somatic:
        sec = cell.soma[i]
        listmech = list(cell.soma[i](0.5))      
        for mech in listmech:
            if str(mech) == 'StochKv':
                print (sec, mech, i)
        i=i+1

    i=0
    for secs in cell.basal:
        sec = cell.dend[i]
        listmech = list(cell.dend[i](0.5))      
        for mech in listmech:
            if str(mech) == 'StochKv':
                print (sec, mech, i)
        i=i+1

    i=0
    for secs in cell.apical:
        sec = cell.apic[i]
        listmech = list(cell.apic[i](0.5))      
        for mech in listmech:
            if str(mech) == 'StochKv':
                print (sec, mech, i)
        i=i+1

    i=0
    for secs in cell.axonal:
        sec = cell.axon[i]
        listmech = list(cell.axon[i](0.5))      
        for mech in listmech:
            if str(mech) == 'StochKv':
                print (sec, mech, i)
        i=i+1     
    
    print (cell)
    return cell

In [7]:
StochKvcells = []
nonStochKvcells = []

for cellName in MEtypePop:
    number = 1
    os.chdir('/home/fernando/S1_BBP/cell_data/'+cellName+'_'+str(number)+'/')
    
    foldermech = os.listdir('mechanisms/')
    if 'StochKv.mod' in foldermech:
        StochKvcells.append(cellName) 
    else:
        nonStochKvcells.append(cellName) 

In [8]:
cellName = StochKvcells[0]
number = 1
cellTemplateName = loadTemplateName(cellName,number)
print(cellName,number,cellTemplateName)
os.chdir('/home/fernando/S1_BBP/cell_data/'+cellName+'_'+str(number)+'/')
# loadCell(cellName,cellTemplateName,number)

L1_HAC_cIR216 1 cIR216_L1_HAC_84f0f8f321


In [9]:
StochKvcellsNumber = 0
for metype in StochKvcells:    
    StochKvcellsNumber = StochKvcellsNumber + MEtypeNumber[metype]

print('cells with StochKv channel = %d (%.2f percent) ' % (StochKvcellsNumber,100.0*StochKvcellsNumber/N))     

cells with StochKv channel = 1137 (3.63 percent) 


# Adaptation to Mouse Model 
https://www.humanbrainproject.eu/en/brain-simulation/mouse-sscx/

In [10]:
def volume(sizey, radius):    
    sizey = 0.001 * sizey # from um to mm
    radius = 0.001 * radius # from um to mm    
    vol = np.pi * radius**2 * sizey
    return vol

def cellNumber(sizey, radius, density):
    number = volume(sizey, radius) * density
    return number

In [11]:
# # RAT Cell 2015 
# Layerthicknesses = {}  
# Layerthicknesses['L1'] = 165
# Layerthicknesses['L2'] = 149
# Layerthicknesses['L3'] = 353
# Layerthicknesses['L4'] = 190
# Layerthicknesses['L5'] = 525
# Layerthicknesses['L6'] = 700
# Neurondensities = {}
# Neurondensities['L1'] = 14200
# Neurondensities['L2'] = 164600
# Neurondensities['L3'] = 83800
# Neurondensities['L4'] = 177300
# Neurondensities['L5'] = 83900
# Neurondensities['L6'] = 131500
# neuronsperlayer = {}
# neuronsperlayer['L1'] = 338
# neuronsperlayer['L23'] = 7524
# neuronsperlayer['L4'] = 4656
# neuronsperlayer['L5'] = 6114
# neuronsperlayer['L6'] = 12651

# radius = 210
# for layer in ['L1','L4','L5','L6']:
#     print ('cell Number in Layer %s =  %.0f' % (layer,cellNumber(Layerthicknesses[layer], radius, Neurondensities[layer])))    
#     print ('comparation BBPwebsite %.3f' % (cellNumber(Layerthicknesses[layer], radius, Neurondensities[layer])/neuronsperlayer[layer]))

# print ('cell Number in Layer L23 =  %.0f' % (cellNumber(Layerthicknesses['L2'], radius, Neurondensities['L2']) + cellNumber(Layerthicknesses['L3'], 210, Neurondensities['L3'])))      
# print ('comparation BBPwebsite %.3f' % ((cellNumber(Layerthicknesses['L2'], radius, Neurondensities['L2']) + cellNumber(Layerthicknesses['L3'], 210, Neurondensities['L3']))/neuronsperlayer['L23']))

# synapticdensities = {}
# synapticdensities['L1'] = 61728000.0
# synapticdensities['L2'] = 101313777.77778
# synapticdensities['L3'] = 101313777.77778
# synapticdensities['L4'] = 351032889.0
# synapticdensities['L5'] = 350586074.0
# synapticdensities['L6'] = 105211259.0
# synapticNumberMouse = 0
# synapticNumberperLayerMouse = {}
# for layer in ['L1','L2','L3','L4','L5','L6']:
#     print ('synaptic Number in Layer %s =  %.0f' % (layer,cellNumber(Layerthicknesses[layer], radius, synapticdensities[layer])))       
#     synapticNumberperLayerMouse[layer] = cellNumber(Layerthicknesses[layer], radius, synapticdensities[layer])
#     synapticNumberMouse = synapticNumberMouse + synapticNumberperLayerMouse[layer]
    
# print ('synaptic Number in a cilinder (d = %d um) =  %.0f' % (2*radius,synapticNumberMouse))   


In [12]:
#  mock circuit DMT neuro_dmt-library/models/mock/circuit/test
# __thickness_layer ={    1: 122.3,    2: 113.5,    3: 302.9,    4: 176.4,    5: 477.9,    6: 647.3}
# __cell_density_layer ={    1: 18566,    2: 131561,    3: 81203,    4: 183128,    5: 98080,    6: 116555}

# Layerthicknesses = {}  # MOUSE
# Layerthicknesses['L1'] = 122.3
# Layerthicknesses['L2'] = 113.5
# Layerthicknesses['L3'] = 302.9
# Layerthicknesses['L4'] = 176.4
# Layerthicknesses['L5'] = 477.9
# Layerthicknesses['L6'] = 647.3
# Neurondensities = {}
# Neurondensities['L1'] = 18566
# Neurondensities['L2'] = 131561
# Neurondensities['L3'] = 81203
# Neurondensities['L4'] = 183128
# Neurondensities['L5'] = 98080
# Neurondensities['L6'] = 116555
# neuronsperlayer = {}
# neuronsperlayer['L1'] = 752/7 
# neuronsperlayer['L2'] = 5893/7
# neuronsperlayer['L3'] = 5340/7
# neuronsperlayer['L4'] = 8204/7
# neuronsperlayer['L5'] = 9711/7
# neuronsperlayer['L6'] = 15991/7

# radius = 125
# for layer in ['L1','L2','L3','L4','L5','L6']:
#     print ('cell Number in Layer %s =  %.0f' % (layer,cellNumber(Layerthicknesses[layer], radius, Neurondensities[layer])))    
#     print ('comparation BBPwebsite %.3f' % (cellNumber(Layerthicknesses[layer], radius, Neurondensities[layer])/neuronsperlayer[layer]))

In [13]:
# neuro_dmt-library/data/sscx_mouse/
# layer_thickness_means: 122.7050 - 96.4912 - 175.8397 - 185.2523 - 362.2625 - 436.2220 (DeFelipe20180212.yaml) 
# density_means: 18.3092  - 141.745  - 73.4301  - 105.262  - 65.955  - 89.3954  (Keller2018Feb14.yaml)  
Layerthicknesses = {}  
Layerthicknesses['L1'] = 122.7
Layerthicknesses['L2'] = 96.5
Layerthicknesses['L3'] = 175.8
Layerthicknesses['L4'] = 185.3
Layerthicknesses['L5'] = 362.3
Layerthicknesses['L6'] = 436.2
Neurondensities = {}
Neurondensities['L1'] = 18309.2
Neurondensities['L2'] = 141745
Neurondensities['L3'] = 73430.1
Neurondensities['L4'] = 105262
Neurondensities['L5'] = 65955
Neurondensities['L6'] = 89395.4

# from https://bbp.epfl.ch/public/simulationapp/index.html#/circuits/sscx_hbp_sa_mouse_microcircuit
neuronsperlayer = {} 
neuronsperlayer['L1'] = 752/7
neuronsperlayer['L2'] = 5893/7
neuronsperlayer['L3'] = 5340/7
neuronsperlayer['L4'] = 8204/7
neuronsperlayer['L5'] = 9711/7
neuronsperlayer['L6'] = 15991/7

radius = 150
cellNumberMouse = 0
y = 0
cellNumberperLayerMouse = {}
for layer in ['L1','L2','L3','L4','L5','L6']:    
    y = y + Layerthicknesses[layer]     
    print ('cell Number in Layer %s =  %.0f' % (layer,cellNumber(Layerthicknesses[layer], radius, Neurondensities[layer])))       
    cellNumberperLayerMouse[layer] = cellNumber(Layerthicknesses[layer], radius, Neurondensities[layer])
    cellNumberMouse = cellNumberMouse + cellNumberperLayerMouse[layer]
    #     print ('comparation BBPwebsite %.3f' % (cellNumber(Layerthicknesses[layer], radius, Neurondensities[layer])/neuronsperlayer[layer]))

cellNumberperLayerMouse['L23'] = cellNumberperLayerMouse['L2'] + cellNumberperLayerMouse['L3']
print ('cell Number in Layer L23 =  %.0f' % cellNumberperLayerMouse['L23'])  
    
print ('cell Number in a cilinder (d = %d um and v = %.3f um^3) =  %.0f' % (2*radius,volume(y,radius),cellNumberMouse))   

# De Felipe et al. (2018).  'Synapse Densities per Layer'
# animal: "mouse" brainArea: "S1 hindlimb"
synapticdensities = {}
synapticdensities['L1'] = 1.664615 * 10**9 # from um^3 to mm^3
synapticdensities['L2'] = 1.340773 * 10**9
synapticdensities['L3'] = 1.289432 * 10**9
synapticdensities['L4'] = 1.499984 * 10**9
synapticdensities['L5'] = 1.183901 * 10**9
synapticdensities['L6'] = 0.904814 * 10**9
synapticdensitiesE = {}
synapticdensitiesE['L1'] = 1.52288128 * 10**9 # from um^3 to mm^3
synapticdensitiesE['L2'] = 1.24516915 * 10**9
synapticdensitiesE['L3'] = 1.18536133 * 10**9
synapticdensitiesE['L4'] = 1.34310283 * 10**9
synapticdensitiesE['L5'] = 1.088029 * 10**9
synapticdensitiesE['L6'] = 0.82219090 * 10**9
synapticdensitiesI = {}
synapticdensitiesI['L1'] = 0.14173376 * 10**9 # from um^3 to mm^3
synapticdensitiesI['L2'] = 0.09560400 * 10**9
synapticdensitiesI['L3'] = 0.10407120 * 10**9
synapticdensitiesI['L4'] = 0.15688177 * 10**9
synapticdensitiesI['L5'] = 0.09686697 * 10**9
synapticdensitiesI['L6'] = 0.08262308 * 10**9

synapticNumberMouse = 0
synapticNumberperLayerMouse = {}
for layer in ['L1','L2','L3','L4','L5','L6']:
    print ('synaptic Number in Layer %s =  %.0f' % (layer,cellNumber(Layerthicknesses[layer], radius, synapticdensities[layer])))       
    synapticNumberperLayerMouse[layer] = cellNumber(Layerthicknesses[layer], radius, synapticdensities[layer])
    synapticNumberMouse = synapticNumberMouse + synapticNumberperLayerMouse[layer]  
print ('synaptic Number in a cilinder (d = %d um) =  %.0f' % (2*radius,synapticNumberMouse))   
print ('synapses per Neuron ~  %.1f' % (synapticNumberMouse/cellNumberMouse))   

synapticNumberMouse = 0
synapticNumberperLayerMouse = {}
for layer in ['L1','L2','L3','L4','L5','L6']:
    print ('synaptic Number in Layer %s =  %.0f' % (layer,cellNumber(Layerthicknesses[layer], radius, synapticdensitiesE[layer])))       
    synapticNumberperLayerMouse[layer] = cellNumber(Layerthicknesses[layer], radius, synapticdensitiesE[layer])
    synapticNumberMouse = synapticNumberMouse + synapticNumberperLayerMouse[layer]  
print ('synaptic EXC Number in a cilinder (d = %d um) =  %.0f' % (2*radius,synapticNumberMouse))   
print ('synapses EXC per Neuron ~  %.1f' % (synapticNumberMouse/cellNumberMouse))   

synapticNumberMouse = 0
synapticNumberperLayerMouse = {}
for layer in ['L1','L2','L3','L4','L5','L6']:
    print ('synaptic Number in Layer %s =  %.0f' % (layer,cellNumber(Layerthicknesses[layer], radius, synapticdensitiesI[layer])))       
    synapticNumberperLayerMouse[layer] = cellNumber(Layerthicknesses[layer], radius, synapticdensitiesI[layer])
    synapticNumberMouse = synapticNumberMouse + synapticNumberperLayerMouse[layer]  
print ('synaptic INH Number in a cilinder (d = %d um) =  %.0f' % (2*radius,synapticNumberMouse))   
print ('synapses INH per Neuron ~  %.1f' % (synapticNumberMouse/cellNumberMouse))   

cell Number in Layer L1 =  159
cell Number in Layer L2 =  967
cell Number in Layer L3 =  912
cell Number in Layer L4 =  1379
cell Number in Layer L5 =  1689
cell Number in Layer L6 =  2756
cell Number in Layer L23 =  1879
cell Number in a cilinder (d = 300 um and v = 0.097 um^3) =  7862
synaptic Number in Layer L1 =  14437459
synaptic Number in Layer L2 =  9145658
synaptic Number in Layer L3 =  16023217
synaptic Number in Layer L4 =  19646918
synaptic Number in Layer L5 =  30319087
synaptic Number in Layer L6 =  27898276
synaptic Number in a cilinder (d = 300 um) =  117470614
synapses per Neuron ~  14941.0
synaptic Number in Layer L1 =  13208181
synaptic Number in Layer L2 =  8493527
synaptic Number in Layer L3 =  14729975
synaptic Number in Layer L4 =  17592075
synaptic Number in Layer L5 =  27863855
synaptic Number in Layer L6 =  25350744
synaptic EXC Number in a cilinder (d = 300 um) =  107238357
synapses EXC per Neuron ~  13639.6
synaptic Number in Layer L1 =  1229278
synaptic Numb

In [14]:
volume(y,radius)

0.09746162889231616

In [15]:
sizeY = 0
for layer in ['L1','L2','L3','L4','L5','L6']:
    sizeY = sizeY + Layerthicknesses[layer]
sizeY

1378.8

In [16]:
# Load 55 Morphological Names and Cell pop numbers -> L1:6 L23:10 L4:12 L5:13 L6:14
# Load 207 Morpho-electrical Names used to import the cells from 'cell_data/' -> L1:14 L23:43 L4:46 L5:52 L6:52
RatL1Number = 0
for popName in MtypePop[0:6]:
    RatL1Number = RatL1Number + MtypeNumber[popName]
RatL23Number = 0
for popName in MtypePop[6:16]:
    RatL23Number = RatL23Number + MtypeNumber[popName]
RatL4Number = 0
for popName in MtypePop[16:28]:
    RatL4Number = RatL4Number + MtypeNumber[popName]
RatL5Number = 0
for popName in MtypePop[28:41]:
    RatL5Number = RatL5Number + MtypeNumber[popName]    
RatL6Number = 0
for popName in MtypePop[41:55]:
    RatL6Number = RatL6Number + MtypeNumber[popName]  

In [17]:
cellNumberperLayerRat = {}
cellNumberperLayerRat['L1'] = RatL1Number
cellNumberperLayerRat['L23'] = RatL23Number
cellNumberperLayerRat['L4'] = RatL4Number
cellNumberperLayerRat['L5'] = RatL5Number
cellNumberperLayerRat['L6'] = RatL6Number
    
RatL1Number + RatL23Number + RatL4Number + RatL5Number + RatL6Number

31346

In [18]:
# N = 0
# MtypeNumberMouse = {}
# for popName in MtypePop:    
    
#     layer = popName[0:2]
    
#     if layer == 'L2':
#         ratio = cellNumberperLayerMouse['L23']/cellNumberperLayerRat['L23']
#     else:
#         ratio = cellNumberperLayerMouse[layer]/cellNumberperLayerRat[layer]

#     MtypeNumberMouse[popName] = int(ratio*MtypeNumber[popName]+0.5)
#     N = N + MtypeNumberMouse[popName]
    
#     print ('cell Number Mouse = %d (%s)' % (int(ratio*MtypeNumber[popName]+0.5),popName))    
# print ('Number of cells = %d' % N)

In [19]:
N = 0
MEtypeNumberMouse = {}
for popName in MEtypePop:    
    
    layer = popName[0:2]
    
    if layer == 'L2':
        ratio = cellNumberperLayerMouse['L23']/cellNumberperLayerRat['L23']
    else:
        ratio = cellNumberperLayerMouse[layer]/cellNumberperLayerRat[layer]

    MEtypeNumberMouse[popName] = int(ratio*MEtypeNumber[popName]+0.5)
    if MEtypeNumberMouse[popName] == 0:
        MEtypeNumberMouse[popName] = 1     
    N = N + MEtypeNumberMouse[popName]
    
#     if popName in StochKvcells:  
#         print ('cell Number Mouse = %d (%s)' % (MEtypeNumberMouse[popName],popName))    
    
print ('Number of cells = %d' % N)

Number of cells = 7859


In [20]:
N = 0
MtypeNumberMouse = {}
for popName in MtypePop:    
    MtypeNumberMouse[popName] = 0

for line in metype_content.split('\n')[:-1]:
    metype, mtype, etype, n, m = line.split()    
   
    MtypeNumberMouse[mtype] = MtypeNumberMouse[mtype] + MEtypeNumberMouse[metype]
    
    N = N + MEtypeNumberMouse[metype]
    
print ('Number of cells = %d ' % (N))

Number of cells = 7859 


In [21]:
StochKvcellsNumber = 0
for metype in StochKvcells:    
    StochKvcellsNumber = StochKvcellsNumber + MEtypeNumberMouse[metype]

print('cells with StochKv channel = %d (%.2f percent) ' % (StochKvcellsNumber,100.0*StochKvcellsNumber/N))     

cells with StochKv channel = 282 (3.59 percent) 


In [22]:
SBCetype = {}

for line in metype_content.split('\n')[:-1]:
    metype, mtype, etype, n, m = line.split()    
    if 'SBC' in metype:
        SBCetype[etype] = 0
        
for line in metype_content.split('\n')[:-1]:
    metype, mtype, etype, n, m = line.split()    
    if 'SBC' in metype:
        SBCetype[etype] = SBCetype[etype] + MEtypeNumberMouse[metype]
        print('%s %s %s %d %d' % (metype, mtype, etype, MEtypeNumberMouse[metype], MtypeNumberMouse[mtype]))
        
SBCetype       

L23_SBC_bNAC219 L23_SBC bNAC219 15 41
L23_SBC_cACint209 L23_SBC cACint209 15 41
L23_SBC_dNAC222 L23_SBC dNAC222 11 41
L4_SBC_bNAC219 L4_SBC bNAC219 6 18
L4_SBC_cACint209 L4_SBC cACint209 7 18
L4_SBC_dNAC222 L4_SBC dNAC222 5 18
L5_SBC_bNAC219 L5_SBC bNAC219 2 6
L5_SBC_cACint209 L5_SBC cACint209 2 6
L5_SBC_dNAC222 L5_SBC dNAC222 2 6
L6_SBC_bNAC219 L6_SBC bNAC219 5 14
L6_SBC_cACint209 L6_SBC cACint209 5 14
L6_SBC_dNAC222 L6_SBC dNAC222 4 14


{'bNAC219': 28, 'cACint209': 29, 'dNAC222': 22}

In [23]:
SBCetype

{'bNAC219': 28, 'cACint209': 29, 'dNAC222': 22}

In [28]:
for metype in MtypePop:
    if metype in Epops:
        print ('%s' % (metype))

L23_PC
L4_PC
L4_SP
L4_SS
L5_STPC
L5_TTPC1
L5_TTPC2
L5_UTPC
L6_BPC
L6_IPC
L6_TPC_L1
L6_TPC_L4
L6_UTPC
